# CS3300 - Final Project
### By: Neel Desai and Tyler Faulkner
### Due: February 21, 2021

## Hypothesis

Some of the questions we are aiming to answer with this notebook are:

Do any variables correalate to if the student gets placed or not?

How well does a predictive classification model perform using correlated variables?

How does that model compare to models using other selections of variables?

If variables in the data set show correlation to the student's placement after college, then a relatively accurate predictive model can be trained to predict a student's placement post-college.


## Data Set

## Data Preprocessing

## Data Analysis and Visualization

## Data Modeling and Prediction

## Results Analysis